<a href="https://www.kaggle.com/code/dankurland/rnsa-dk-and-npd-working-model-v2?scriptVersionId=120659114" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Install two dedicated libraries for handling jpeg2000 files
!pip install -qU python-gdcm pylibjpeg

#Install dicomsdl for handling dicom files a little faster
!pip install -qU dicomsdl

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import numpy.random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for making plots
from tqdm.notebook import tqdm
import sys
from multiprocessing import cpu_count
import cv2
import ray #Used for parallel processing quickly
import time
from sklearn.model_selection import train_test_split
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        (os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import csv data as a data frame.
csv_train = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/train.csv")
csv_test = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/test.csv")

csv_train.info()
# age, BIRADS, and density are missing some entries; the latter two are missing many.

In [ ]:
# For a preliminary attempt, let's drop rows that are not even in the test set.
columns_in_test = []
for col in csv_test.columns:
    if col != 'prediction_id': # This column is not in the training set.
        columns_in_test.append(col)
columns_in_test.append('cancer') # Make sure we don't drop the target column yet!
print(columns_in_test)

data_train = csv_train[columns_in_test]
# data_train.head()

#One single stupid file is completely blank for no reason: patient 822 image 1942326353. I manually remove it
bad_ids = [1942326353, 1078943060, 398038886, 439796429, 63473691]

data_train = data_train.drop(data_train[data_train['image_id'].isin(bad_ids)].index)
data_train.shape

# Get rid of entries with missing age, since there's only a small number of these.
data_train = data_train.dropna(subset=['age'])
# Let's rescale so that ages listed are between (0, 1) for CNN convenience.
data_train['age'] = data_train['age'] / data_train['age'].max()

In [ ]:
# Create a list of all cancer patients and non cancer patients.
cancer_train = data_train.loc[csv_train.cancer == 1]
nocancer_train = data_train.loc[csv_train.cancer == 0]

cancer_train.head()

In [ ]:
# The PNGs were finally output successfully! They are now saved & imported to Input.
# Time to start training on the data.

# Converting all the PNGs back to np.array's & adding relevant data from
# the CSV file entries for each image.
# According to https://www.kaggle.com/code/devashishprasad/fastest-way-to-read-and-process-images
# PIL has a pretty fast image importing process.
import PIL
from PIL import Image

# Parallel processing iterator. Runs "ray.get()" on each ray future.
def to_iterator(obj_ids):
    while obj_ids:
        done,  obj_ids = ray.wait(obj_ids)
        yield ray.get(done[0])

#Getting one png
def import_PNG(imagepath):
    im = (np.array( Image.open(imagepath)) / 255.  )
    return im

@ray.remote
def create_info_array(imagepath, csv_info):
    
    image_id = imagepath.split('/')[-1].split('.')[0] #get image id from filename
    image_id = int(image_id)
    csv_image_data = csv_info.loc[csv_info['image_id'] == image_id] #get proper row
    # we don't need the patient_id or image_id, it carries no clinical information
    csv_image_data = csv_image_data.drop(['patient_id', 'image_id'], axis=1) 
    
    image_data = list(csv_image_data.values.tolist()[0]) #convert to list
    image = import_PNG(imagepath) #get the image as an np array
    return image, image_data

def onehot(df, encode):
    temp = pd.get_dummies(df[encode])
    df = df.drop([encode], axis=1)
    df = pd.concat([df, temp], axis=1)
    return df

# For now let's assume these are not relevant; I find it unlikely that location has that much relevance here.
# Some areas may be e.g. more affluent than others which could in principle affect results, but we have
# little information about the sites or machines to deduce anything meaningful.
irrelevant_col = ['machine_id', 'site_id'] 
data_train_relevant = data_train.drop(irrelevant_col, axis=1)

# One-Hot: the Laterality and View columns are strings; there's only a few types of entries for each so let's just one-hot encode them.
onehotcols = ['laterality', 'view']
for col in onehotcols:
    data_train_relevant = onehot(data_train_relevant, col)

# Implant column is a int64 for some reason; let's convert it to uint8 so it's lighter.
data_train_relevant['implant'] = data_train_relevant['implant'].astype('uint8')

#Now we have all of the csv data in numerical values. We can input it to a network easily.

pngfolder = '/kaggle/input/rnsa-s-m-pngs-v2/train/processed_512'

In [ ]:
#g = glob.iglob(pngfolder + '/**/*.png', recursive=True) #this was to test some weird error, commenting it out
#for i in range(10):
#    print(next(g))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

with tf.device('/gpu:0'):
    # convoluting and pooling image - convolution less consequential than pooling for image dimension
    
    input_Image = tf.keras.layers.Input(shape=(512,512, 1))
    print(type(input_Image))
    
    # randomly rotate image
    iput_Image = tf.keras.layers.RandomRotation((-0.4, 0.4))(input_Image)
    
    print(type(iput_Image))
    
    
    print(iput_Image.get_shape())
    
    # relu is a transformation applied to the output of a node
    conv_Image = Conv2D(3, 13, activation='relu', 
                        input_shape = (512, 512, 1))(iput_Image) # convolution neural network

    
    
    print(conv_Image.get_shape())
    
    # we take the max of different sections of the image, similar to the concept of a kernal
    # my gut is that pool size can be much smaller. We can test out a few different iterations
    pool_Image = MaxPooling2D(pool_size=10)(conv_Image) # to reduce the size of the image for input to the rest of the ML
                        
   
    
    print(pool_Image.get_shape())
    
#     seems to be 25, 25, 10 shape

#   second round of convolutions / pooling 

    conv_Image = Conv2D(3, 5, activation='relu', 
                        input_shape = (50, 50, 3))(pool_Image)
    
    print(conv_Image.get_shape())

    
    pool_Image = MaxPooling2D(pool_size=4)(conv_Image)
    
    print(pool_Image.get_shape())
    
    
    
    flatten_Image = Flatten()(pool_Image)
    
    # concating the input tags to the image
    input_Tags = Input(shape=(10,))
    input_all = Concatenate()([flatten_Image, input_Tags])
    
    
    # added more nodes per layer and more layers
    CNN = GaussianNoise(0.1)(input_all)
    CNN = Dense(10, activation='relu')(CNN)
#     CNN = Dense(10, activation='relu')(CNN)
#     CNN = Dense(10, activation='relu')(CNN)
    CNN = Dense(10, activation='relu')(CNN)
    CNN = Dense(1, activation='sigmoid')(CNN)


    ML = Model(inputs=[input_Image, input_Tags], outputs=CNN)
    ML.summary()
    
    # seems to be the right loss function - used for binary outcomes
    # I set from_logits to True because I'm not assuming that things are standardized. It generally seems like a safer option
    
    # decreased learning rate significantly to avoid immediately finding local minimum
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.01)
    
    ML.compile(optimizer=optimizer,
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
               metrics=['accuracy']
              )
# This ML model appears to be really, really inefficient/slow (?)

In [ ]:
# defining class weights to mitigate bias 

df_cancer = data_train_relevant.loc[data_train_relevant['cancer']==1]
pos = len(df_cancer.index)
df_clear = data_train_relevant.loc[data_train_relevant['cancer']==0]
neg = len(df_clear.index)
total = (pos + neg)
print(pos, neg, total)

weight_for_0 = .6
weight_for_1 = .5

# weight_for_0 = (50 / neg) * (total / 2.0)
# weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
# def fracture_dataframe(df, samplefrac = 1, samplefrac_clear = 1, seed = None, shuffle = False, signal_frac = None):
#     out = []
    
#     if seed != None:
#         np.random.seed(seed)
#     df_cancer = df.loc[df['cancer']==1]
#     df_clear = df.loc[df['cancer']==0]
#     if signal_frac == None:
#         signal_frac = 1
#     N_cancer = len(df_cancer)
#     N_clear = len(df_clear)
#     num_arrays = int(np.ceil(N_cancer / (N_clear/signal_frac)))
    
#     df_cancer.sample(frac=samplefrac).reset_index(drop=True, inplace=True)
#     df_cancer.sample(frac=samplefrac_clear).reset_index(drop=True, inplace=True)
#     cancer_subarrays = np.array_split(df_cancer, num_arrays)
    
#     out = []
#     for df_frac in cancer_subarrays:
#         pd_merge = pd.concat([df_frac, df_clear])
#         pd_merge.sample(frac=samplefrac).reset_index(drop=True, inplace=True)
#         out.append(pd_merge)
#     return out

In [ ]:
# split test and train
def split_test_train(df, samplefrac = 1, test_size = 1000):
    df.sample(frac=samplefrac).reset_index(drop=True, inplace=True)
    df_bottom = df.iloc[-test_size: ]
    df = df.iloc[:-test_size]
    test = df_bottom
    return test, df
    

# split train into batches
def split_dataframe(df, train_sample = 50, samplefrac = 1):        
    df_cancer = df.loc[df['cancer']==1].iloc[:train_sample]
    df_clear = df.loc[df['cancer']==0].iloc[:train_sample]
    train = pd.concat([df_cancer, df_clear])    
    train.sample(frac=samplefrac).reset_index(drop=True, inplace=True)
    return train

In [ ]:
import sklearn.utils

# We need to manually shuffle the data_frame because 
# the keras validation split functionality doesn't do this!
data_train_relevant =  data_train_relevant.sample(frac=1).reset_index(drop=True)

# We'll add a column to the dataframe with all of the filenames to use with flow_from_dataframe.
data_train_relevant['file'] = data_train_relevant.apply(
    lambda x: pngfolder + '/' + str(int(x['patient_id'])) + '/' + str(int(x['image_id'])) 
    + '.png', axis=1)

#print(data_train_relevant.head)

# To create the correct shape of inputs for the network, we need to get both the
# image data and the csv data -- have all the columns be the "target y" for flow_from_dataframe.
# this function grabs the pngs (from file path 'file' column and the related data in the DF. The image imports happen here)
#split the sample into training and validation sets


data_test, training_data = split_test_train(data_train_relevant, samplefrac = 1, test_size = 5000)


# To create the correct shape of inputs for the network, we need to get both the
# image data and the csv data -- have all the columns be the "target y" for flow_from_dataframe.
def data_from_dataframe(directory, generator, subset='training', batch_size = 32, shuffle=True,
                        data = data_train_relevant, columns = ['cancer'], seed = None):
    
    if subset == 'test':
        subset2 = 'training'
    else:
        subset2 = subset
    
    gendat = generator.flow_from_dataframe(data, directory=directory, shuffle = shuffle, 
                                           target_size = (ImageSize, ImageSize),
                                           subset = subset2, batch_size = batch_size,
                                           x_col = 'file', y_col = columns, 
                                           class_mode = 'multi_output', 
                                           color_mode = 'grayscale', 
                                           validate_filenames=False)
    
    i = 0
    N = gendat.n
    while i < N:
        data = gendat.next()
        x_im = np.array(data[0]).astype(np.float16) # reference to image
        x_info = np.array(data[1][:-1]).T.astype(np.float16) # data columns
        y = data[1][-1]  # target column (i.e. cancer)
        i += batch_size
        if subset == 'test':
            yield [x_im, x_info]
        else:
            yield [x_im, x_info], y


ImageSize = 512 # i don't know if we should be resizing this
val_split = 0.2 
batch_size = 32 # in order to conserve memory, maybe we should reduce this

cols = ['age', 'implant', 'L', 'R', 'AT', 'CC', 'LM', 'LMO', 'ML', 'MLO', 'cancer']
# Make a Keras ImageDataGenerator
imagegen = ImageDataGenerator(rescale = 1/255, validation_split = val_split)
imagegen2 = ImageDataGenerator(rescale = 1/255, validation_split = 0)

# batch size just has to do with iterating through the dataframe - every epoch in our ml model iterates through all the batches


In [ ]:
#xtest = next(train_gen)
# this was for testing to debug a weird error that was coming up

In [ ]:
import gc

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# don't know if we need checkpointer
# checkpointer = ModelCheckpoint(filepath='weights.hdf5', 
#                                verbose=1, save_best_only=True)

#  why is validation data a tuple and train gen isn't?
# CHANGE EPOCHS TO SOMETHING REASONABLE

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5)


for counter in range(4000):
    current_batch = split_dataframe(training_data, train_sample = 50)
    train_gen = data_from_dataframe(None, imagegen, subset='training', batch_size = batch_size,
                                data = current_batch, columns = cols)
    val_gen = data_from_dataframe(None, imagegen, subset='validation', batch_size = batch_size,
                                data = current_batch, columns = cols)
    ML.fit(train_gen, epochs=1, validation_data = val_gen, shuffle = True, class_weight = class_weight,
           validation_steps = 1, batch_size = batch_size, 
           workers = 1, use_multiprocessing = False, 
           steps_per_epoch = (len(data_train_relevant) * (1-val_split)) // batch_size,
           callbacks = [early_stopping])
    del train_gen
    del val_gen
    del current_batch
#     gc.collect(0)
#     gc.collect(1)
#     gc.collect(2)

In [ ]:
ML.save_weights('breastcancerpredictor.h5')

In [ ]:
test_gen = data_from_dataframe(None, imagegen2, batch_size = batch_size, subset='test',
                                data = data_test, columns = cols, shuffle = False)
#ML.evaluate(test_gen, batch_size = batch_size, workers = cpu_count(), 
#            use_multiprocessing = True, steps = (len(data_test) // batch_size))
y = ML.predict(test_gen, steps = np.ceil(len(data_test) / batch_size), verbose=1)
print(y)
y = y.T[0]
print(y)

In [ ]:
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [ ]:
labels_true = np.array(data_test['cancer'])
score_1 = pfbeta(labels_true, y)
print(score_1)

In [ ]:
# plt.hist(y)
plt.hist(np.array(y))

In [ ]:
plt.hist(y-labels_true[:len(y)])

In [ ]:
# want to train our model in separate iterations without each round influencing the next
del ML